In [2]:
import pandas as pd
import pygsheets
# import os
from datetime import datetime, date
# import sys

import gsheet_utility as gsheetUtility
# import Email_Code as fkEmail
import data_processing as data_processing
import numpy as np

In [5]:
basePath = '/Users/a/Documents/GitHub/exception_visibility/'

today = date.today()
today = str(today.year) + "-" + str('{:02d}'.format(today.month)) + "-" + str('{:02d}'.format(today.day))
spf_raw_data_location = basePath + 'ETL/SPF_Raw_Data/'
pv_raw_data_location = basePath + 'ETL/PV_Raw_Data/'
audit_raw_data_location = basePath + 'ETL/Audit_Raw_Data/'
spfsheetdatapath = basePath + 'ETL/gsheet_data/spf_data/' + today + '.csv'
pvsheetdatapath = basePath + 'ETL/gsheet_data/pv_data/' + today + '.csv'
auditsheetdatapath = basePath + 'ETL/gsheet_data/audit_data/' + today + '.csv'

html_message=''
error_message=''

date_master_file = pd.read_csv( '/Users/a/Documents/GitHub/exception_visibility/Dashboard/data/week_details.csv')
date_master_file['weekday'] = pd.to_datetime(date_master_file["weekday"],format = '%d/%m/%Y').dt.date
date_master_file['month_year'] = pd.to_datetime(date_master_file["month_year"],format = '%d/%m/%Y').dt.date
orphangsheetdatapath = basePath + 'ETL/gsheet_data/orphan_data/' + today + '.csv'
hub_zone_data = gsheetUtility.get_gsheet_data('https://docs.google.com/spreadsheets/d/187u3lIk3GSDiHuUm-ZLno_keTe-jCAu1bqiTon7ExgU/edit?usp=sharing', 'A3', orphangsheetdatapath,'Hubdetails',0)

02/22/2022, 05:27:35 Starting Fetching Gsheet Data
02/22/2022, 05:27:40 Fetching Gsheet Completed
Hubdetails
A3


In [91]:
try:
    html_message = html_message + "Data Ingestion Job for PV data form started at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting pv data process')
    html_message = html_message + "Starting to fetch PV data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting pv data process')
    pvData = gsheetUtility.get_gsheet_data('https://docs.google.com/spreadsheets/d/1jsroohFh1uoPqw-6jCcDztEUYAgukZGwFP0YeKgvhH0/edit?usp=sharing', 'A2', pvsheetdatapath,'PV',0)
    pv_columns = ['sl_no','month','scanned_timestamp','shipment_id','motherhub_name','reason']
    html_message = html_message + "completed fetching pv data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Fetching pv data process Completed')
    pvData = gsheetUtility.assignDBColumns(pvData, gsheet_asset='PV Data', dbColumns=pv_columns)

    pvData = data_processing.datatype_conversion(pvData, 'scanned_timestamp', 'datetime')
    # spfData['scanned_date'] = spfData['scanned_timestamp'].dt.date
    # spfData = data_processing.fetch_date_details(spfData, 'scanned_timestamp', date_master_file)

    pvData.shipment_id = pvData.shipment_id.replace(r'^\s*$', np.NaN, regex=True)
    pvData['is_tracking_id_available'] = ''
    pvData.loc[pvData.shipment_id.isna(), ['is_tracking_id_available']]='No'
    pvData.loc[~pvData.shipment_id.isna(), ['is_tracking_id_available']] = 'Yes'

    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to create pv data files')
    html_message = html_message + "Starting to create pv data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    pvData['shipment_id'] = pvData['shipment_id'].str.replace(',', '')
    # spfData['shipment_id'] = spfData['shipment_id'].str.replace('.0', '')
    pvData.shipment_id = pvData.shipment_id.str.replace(' ', '')
    pvData.shipment_id = pvData.shipment_id.fillna(0)
    # spfData.shipment_id = spfData[1378:1380].shipment_id.str.replace(r'[^0-9]+', '')
    # spfData.shipment_id =pd.to_numeric(spfData.shipment_id)
    pvData.shipment_id = pvData.shipment_id.str.upper()

    pvData.motherhub_name = pvData.motherhub_name.str.upper()
    pvData = pd.merge(pvData, hub_zone_data, left_on='motherhub_name', right_on='hub_name', how='left')

    filenames = []
    created_raw_files = data_processing.fetch_created_files(pv_raw_data_location)
    filenames = data_processing.create_rewrite_raw_files(pvData,'month', created_raw_files, pv_raw_data_location )
    html_message = html_message + "Completed creating pv data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Completed creating pv raw data files')

    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to collate data for pv dashboard')
    # html_message = html_message + "Starting to create spf raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    pv_final_columns = ['sl_no','month','scanned_timestamp','shipment_id','motherhub_name','reason', 'is_tracking_id_available', 'zone','asset']
    created_raw_files = data_processing.fetch_created_files(pv_raw_data_location)
    pv_dashboard_data = data_processing.collate_data_for_dashboard(datetime.now().date(), 180, created_raw_files, pv_raw_data_location, pv_final_columns,'month')
    pv_dashboard_data.to_csv(basePath + 'Dashboard/data/pv_full_data.csv', index=False)
except Exception as ex:
        error_message = datetime.now().strftime("%d-%m-%Y %H:%M:%S") + str(ex.__class__).replace('<','').replace('>','') + datetime.now().strftime("%d/%m/%Y %H:%M:%S") + str(ex).replace('<','').replace('>','')
        print(error_message)
if len(error_message)>0:
    html_message = html_message + "<h2>Error Occured - Data Ingestion Failed for pv <p> Error Message : " + error_message + '</h2><br>'
else:
    html_message = html_message + "Data Ingestion Successfull for PV Data form completed at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'


02/16/2022 14:03:13 Starting pv data process
02/16/2022 14:03:13 Starting pv data process
02/16/2022, 14:03:13 Starting Fetching Gsheet Data
02/16/2022, 14:03:18 Fetching Gsheet Completed
PV
A2
02/16/2022 14:03:18 Fetching pv data process Completed
Converting scanned_timestamp to datetime
Converted scanned_timestamp to datetime
02/16/2022 14:03:18 Starting to create pv data files
fetching names of already created raw files
started creating raw files
creating raw file for the date Apr-21.csv
creating raw file for the date May-21.csv
creating raw file for the date Jun-21.csv
creating raw file for the date Mar-21.csv
creating raw file for the date Jul-21.csv
creating raw file for the date Aug-21.csv
creating raw file for the date Oct-21.csv
creating raw file for the date Nov-21.csv
creating raw file for the date Dec-21.csv
creating raw file for the date Jan-22.csv
copmpleted creating 10 files 
02/16/2022 14:03:18 Completed creating pv raw data files
02/16/2022 14:03:18 Starting to collate

In [4]:
try:
    html_message = html_message + "Data Ingestion Job for spf data form started at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting spf data process')
    html_message = html_message + "Starting to fetch spf data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting spf data process')
    spfData = gsheetUtility.get_gsheet_data('https://docs.google.com/spreadsheets/d/1jsroohFh1uoPqw-6jCcDztEUYAgukZGwFP0YeKgvhH0/edit?usp=sharing', 'A2', spfsheetdatapath,'spf',1)
    spf_columns = ['sl_no','month','scanned_timestamp','shipment_id','motherhub_name','reason']
    html_message = html_message + "completed fetching spf data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Fetching spf data process Completed')
    spfData = gsheetUtility.assignDBColumns(spfData, gsheet_asset='SPF Data', dbColumns=spf_columns)

    spfData = data_processing.datatype_conversion(spfData, 'scanned_timestamp', 'datetime')
    # spfData['scanned_date'] = spfData['scanned_timestamp'].dt.date
    # spfData = data_processing.fetch_date_details(spfData, 'scanned_timestamp', date_master_file)

    spfData.shipment_id = spfData.shipment_id.replace(r'^\s*$', np.NaN, regex=True)
    spfData['is_tracking_id_available'] = ''
    spfData.loc[spfData.shipment_id.isna(), ['is_tracking_id_available']]='No'
    spfData.loc[~spfData.shipment_id.isna(), ['is_tracking_id_available']] = 'Yes'

    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to create orphan raw data files')
    html_message = html_message + "Starting to create spf data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    spfData['shipment_id'] = spfData['shipment_id'].str.replace(',', '')
    # spfData['shipment_id'] = spfData['shipment_id'].str.replace('.0', '')
    spfData.shipment_id = spfData.shipment_id.str.replace(' ', '')
    spfData.shipment_id = spfData.shipment_id.fillna(0)
    # spfData.shipment_id = spfData[1378:1380].shipment_id.str.replace(r'[^0-9]+', '')
    # spfData.shipment_id =pd.to_numeric(spfData.shipment_id)
    spfData.shipment_id = spfData.shipment_id.str.upper()

    spfData.motherhub_name = spfData.motherhub_name.str.upper()
    spfData = pd.merge(spfData, hub_zone_data, left_on='motherhub_name', right_on='hub_name', how='left')

    filenames = []
    created_raw_files = data_processing.fetch_created_files(spf_raw_data_location)
    filenames = data_processing.create_rewrite_raw_files(spfData,'month', created_raw_files, spf_raw_data_location )
    html_message = html_message + "Completed creating spf data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Completed creating spf raw data files')

    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to collate data for SPF dashboard')
    # html_message = html_message + "Starting to create spf raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    spf_final_columns = ['sl_no','month','scanned_timestamp','shipment_id','motherhub_name','reason', 'is_tracking_id_available', 'zone','asset']
    created_raw_files = data_processing.fetch_created_files(spf_raw_data_location)
    spf_dashboard_data = data_processing.collate_data_for_dashboard(datetime.now().date(), 180, created_raw_files, spf_raw_data_location, spf_final_columns,'month')
    spf_dashboard_data = data_processing.dashboard_data_pivot(spf_dashboard_data, pivot_indexes=['month','motherhub_name','is_tracking_id_available','zone','asset'], value_calc_column = 'shipment_id', rename_column_to = 'count', aggregation_func = len)
    spf_dashboard_data.to_csv(basePath + 'Dashboard/data/spf_full_data.csv', index=False)
except Exception as ex:
        error_message = datetime.now().strftime("%d-%m-%Y %H:%M:%S") + str(ex.__class__).replace('<','').replace('>','') + datetime.now().strftime("%d/%m/%Y %H:%M:%S") + str(ex).replace('<','').replace('>','')
        print(error_message)
if len(error_message)>0:
    html_message = html_message + "<h2>Error Occured - Data Ingestion Failed for SPF <p> Error Message : " + error_message + '</h2><br>'
else:
    html_message = html_message + "Data Ingestion Successfull for SPF Data form completed at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'

02/16/2022 07:37:54 Starting spf data process
02/16/2022 07:37:54 Starting spf data process
02/16/2022, 07:37:54 Starting Fetching Gsheet Data
02/16/2022, 07:37:57 Fetching Gsheet Completed
spf
A2
02/16/2022 07:37:57 Fetching spf data process Completed
Converting scanned_timestamp to datetime
Converted scanned_timestamp to datetime
02/16/2022 07:37:57 Starting to create orphan raw data files
fetching names of already created raw files
started creating raw files
creating raw file for the date Mar-21.csv
creating raw file for the date Apr-21.csv
creating raw file for the date May-21.csv
creating raw file for the date Jun-21.csv
creating raw file for the date June-21.csv
creating raw file for the date July-21.csv
creating raw file for the date Aug-21.csv
creating raw file for the date Sep-21.csv
creating raw file for the date Oct-21.csv
creating raw file for the date Nov-21.csv
creating raw file for the date Dec-21.csv
creating raw file for the date Jan-22.csv
copmpleted creating 12 files

In [87]:
try:
    html_message = html_message + "Data Ingestion Job for audit data form started at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting audit data process')
    html_message = html_message + "Starting to fetch audit data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting audit data process')
    auditData = gsheetUtility.get_gsheet_data('https://docs.google.com/spreadsheets/d/1jsroohFh1uoPqw-6jCcDztEUYAgukZGwFP0YeKgvhH0/edit?usp=sharing', 'A2', pvsheetdatapath,'Audit',2)
    audit_columns = ['sl_no','month','scanned_timestamp','motherhub_name','qc_name','shipment_id','product_details','colour','image','brand','size','mrp','remarks','ok_count','not_ok_count','result']
    html_message = html_message + "completed fetching audit data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Fetching audit data process Completed')
    auditData = gsheetUtility.assignDBColumns(auditData, gsheet_asset='Audit Data', dbColumns=audit_columns)
    auditData.drop(columns=['month'], inplace=True)
    auditData = data_processing.datatype_conversion(auditData, 'scanned_timestamp', 'datetime')
    auditData['scanned_date'] = auditData['scanned_timestamp'].dt.date
    auditData = data_processing.fetch_date_details(auditData, 'scanned_date', date_master_file)

    auditData.shipment_id = auditData.shipment_id.replace(r'^\s*$', np.NaN, regex=True)
    auditData['is_tracking_id_available'] = ''
    auditData.loc[auditData.shipment_id.isna(), ['is_tracking_id_available']]='No'
    auditData.loc[~auditData.shipment_id.isna(), ['is_tracking_id_available']] = 'Yes'

    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to create audit data files')
    html_message = html_message + "Starting to create pv data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    auditData['shipment_id'] = auditData['shipment_id'].str.replace(',', '')
    # spfData['shipment_id'] = spfData['shipment_id'].str.replace('.0', '')
    auditData.shipment_id = auditData.shipment_id.str.replace(' ', '')
    auditData.shipment_id = auditData.shipment_id.fillna(0)
    # spfData.shipment_id = spfData[1378:1380].shipment_id.str.replace(r'[^0-9]+', '')
    # spfData.shipment_id =pd.to_numeric(spfData.shipment_id)
    auditData.shipment_id = auditData.shipment_id.str.upper()

    auditData.motherhub_name = auditData.motherhub_name.str.upper()
    auditData = pd.merge(auditData, hub_zone_data, left_on='motherhub_name', right_on='hub_name', how='left')

    auditData.colour= auditData.colour.replace(r'^\s*$', np.NaN, regex=True)
    auditData.colour.fillna('NA', inplace=True)
    auditData.image= auditData.image.replace(r'^\s*$', np.NaN, regex=True)
    auditData.image.fillna('NA', inplace=True)
    auditData.brand= auditData.brand.replace(r'^\s*$', np.NaN, regex=True)
    auditData.brand.fillna('NA', inplace=True)
    auditData.mrp= auditData.mrp.replace(r'^\s*$', np.NaN, regex=True)
    auditData.mrp.fillna('NA', inplace=True)
    for column in ['colour','image', 'brand', 'size','mrp']:
        auditData[column] = auditData[column].str.upper()
    auditData['ok_count'] = auditData[['colour','image', 'brand', 'size','mrp']].apply(lambda x: (1 if (x['colour']=='OK' or x['colour']=='NA') else 0 )+ (1 if (x['image']=='OK' or x['image']=='NA') else 0) +
                                                                    (1 if (x['brand']=='OK' or x['brand']=='NA') else 0 ) + (1 if (x['size']=='OK'  or x['size']=='NA') else 0) + 
                                                                    (1 if (x['mrp']=='OK'  or x['mrp']=='NA') else 0), axis=1)
    auditData['not_ok_count'] = auditData[['colour','image', 'brand', 'size','mrp']].apply(lambda x: (1 if x['colour']=='NOT OK' else 0 )+ (1 if x['image']=='NOT OK' else 0) +
                                                                    (1 if x['brand']=='NOT OK' else 0 ) + (1 if x['size']=='NOT OK' else 0) + + (1 if x['mrp']=='NOT OK' else 0), axis=1)
    auditData['result'] = auditData[['ok_count', 'not_ok_count',]].apply(lambda x: 'Pass' if x['ok_count'] / (x['ok_count'] + x['not_ok_count']) ==1 else 'Fail', axis =1)
    
    filenames = []
    created_raw_files = data_processing.fetch_created_files(audit_raw_data_location)
    filenames = data_processing.create_rewrite_raw_files(auditData,'scanned_date', created_raw_files, audit_raw_data_location )
    html_message = html_message + "Completed creating pv data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Completed creating pv raw data files')

    print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to collate data for audit dashboard')
    # html_message = html_message + "Starting to create spf raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
    audit_final_columns = ['sl_no', 'scanned_timestamp', 'motherhub_name','zone', 'qc_name', 'shipment_id', 'product_details', 'colour', 'image', 'brand', 'size',
                        'mrp', 'remarks', 'ok_count', 'not_ok_count', 'result', 'asset','scanned_date', 'weekend', 'month', 'year', 'month_year', 'weeknum',
                        'is_tracking_id_available' ]
    created_raw_files = data_processing.fetch_created_files(audit_raw_data_location)
    audit_dashboard_data = data_processing.collate_data_for_dashboard(datetime.now().date(), 180, created_raw_files, audit_raw_data_location, audit_final_columns,'month')
    audit_dashboard_data = data_processing.dashboard_data_pivot(audit_dashboard_data, pivot_indexes=['scanned_date','month', 'year', 'motherhub_name', 'zone', 'is_tracking_id_available', 'asset','result'], value_calc_column = 'shipment_id', rename_column_to = 'count', aggregation_func = len)
    audit_dashboard_data.to_csv(basePath + 'Dashboard/data/audit_full_data.csv', index=False)
except Exception as ex:
        error_message = datetime.now().strftime("%d-%m-%Y %H:%M:%S") + str(ex.__class__).replace('<','').replace('>','') + datetime.now().strftime("%d/%m/%Y %H:%M:%S") + str(ex).replace('<','').replace('>','')
        print(error_message)
if len(error_message)>0:
    html_message = html_message + "<h2>Error Occured - Data Ingestion Failed for audit data <p> Error Message : " + error_message + '</h2><br>'
else:
    html_message = html_message + "Data Ingestion Successfull for audit Data form completed at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'

02/16/2022 08:34:17 Starting audit data process
02/16/2022 08:34:17 Starting audit data process
02/16/2022, 08:34:17 Starting Fetching Gsheet Data
02/16/2022, 08:34:20 Fetching Gsheet Completed
Audit
A2
02/16/2022 08:34:20 Fetching audit data process Completed
Converting scanned_timestamp to datetime
Converted scanned_timestamp to datetime
mapping dates and weekend to scanned date
02/16/2022 08:34:20 Starting to create audit data files
fetching names of already created raw files
started creating raw files
creating raw file for the date 2021-09-08.csv
creating raw file for the date 2021-09-11.csv
creating raw file for the date 2021-09-13.csv
creating raw file for the date 2021-09-14.csv
creating raw file for the date 2021-09-15.csv
creating raw file for the date 2021-09-17.csv
creating raw file for the date 2021-09-20.csv
creating raw file for the date 2021-09-23.csv
creating raw file for the date 2021-09-18.csv
creating raw file for the date 2021-10-03.csv
creating raw file for the dat

In [92]:
pv_dashboard_data

,sl_no,month,scanned_timestamp,shipment_id,motherhub_name,reason,is_tracking_id_available,zone,asset
0,19,Jun-21,NaN,FMPC1239597441,MOTHERHUB_JKS,Refinishing,Yes,North,PV Data
1,20,Jun-21,NaN,FMPR0179802346,MOTHERHUB_RCB,Seller return area,Yes,South 1,PV Data
2,21,Jun-21,NaN,FMPP0756032839,MOTHERHUB_RCB,external_liquidation_non_damaged_area,Yes,South 1,PV Data
3,22,Jun-21,NaN,FMPP0781290983,MOTHERHUB_BLR,returns_supplier_return_area,Yes,South 1,PV Data
4,23,Jun-21,NaN,FMPC1214246345,MOTHERHUB_HYP,"fraud area, missing",Yes,South 2,PV Data
...,...,...,...,...,...,...,...,...,...
364,1642,May-21,NaN,FMPC1269421201,MOTHERHUB_JKS_RET,"Missing, fraud",Yes,North,PV Data
365,1643,May-21,NaN,FMPR0200853753,MOTHERHUB_JKS_RET,"Missing, fraud",Yes,North,PV Data
366,1644,May-21,NaN,FMPC1306607675,MOTHERHUB_JKS_RET,"Missing, fraud",Yes,North,PV Data
367,1645,May-21,NaN,FMPC1288755440,MOTHERHUB_JKS_RET,"Missing, fraud",Yes,North,PV Data


In [89]:
data_processing.dashboard_data_pivot(audit_dashboard_data, pivot_indexes=['scanned_date','month', 'year', 'motherhub_name', 'zone', 'is_tracking_id_available', 'asset','result'], value_calc_column = 'shipment_id', rename_column_to = 'count', aggregation_func = len)

# data_processing.collate_data_for_dashboard(datetime.now().date(), 180, created_raw_files, audit_raw_data_location, audit_final_columns,'date')

,scanned_date,month,year,motherhub_name,zone,is_tracking_id_available,asset,result,count
0,2021-09-08,September,2021,MOTHERHUB_BLR,South 1,Yes,Audit Data,Fail,2
1,2021-09-08,September,2021,MOTHERHUB_BLR,South 1,Yes,Audit Data,Pass,6
2,2021-09-08,September,2021,MOTHERHUB_SAI_3,West,Yes,Audit Data,Fail,2
3,2021-09-08,September,2021,MOTHERHUB_SAI_3,West,Yes,Audit Data,Pass,8
4,2021-09-08,September,2021,MOTHERHUB_SNK,East,Yes,Audit Data,Pass,4
...,...,...,...,...,...,...,...,...,...
271,2022-01-24,January,2022,MOTHERHUB_BLR,South 1,Yes,Audit Data,Pass,10
272,2022-01-28,January,2022,MOTHERHUB_HRN,East,Yes,Audit Data,Pass,5
273,2022-01-28,January,2022,MOTHERHUB_HRN,East,Yes,Audit Data,Pass,5
274,2022-01-29,January,2022,MOTHERHUB_GGN,North,Yes,Audit Data,Pass,5


In [83]:
audit_dashboard_data

,sl_no,scanned_timestamp,motherhub_name,zone,qc_name,shipment_id,product_details,colour,image,brand,...,not_ok_count,result,asset,scanned_date,weekend,month,year,month_year,weeknum,is_tracking_id_available


In [75]:
auditData.colour= auditData.colour.replace(r'^\s*$', np.NaN, regex=True)
auditData.colour.fillna('NA', inplace=True)
auditData.image= auditData.image.replace(r'^\s*$', np.NaN, regex=True)
auditData.image.fillna('NA', inplace=True)
auditData.brand= auditData.brand.replace(r'^\s*$', np.NaN, regex=True)
auditData.brand.fillna('NA', inplace=True)
auditData.mrp= auditData.mrp.replace(r'^\s*$', np.NaN, regex=True)
auditData.mrp.fillna('NA', inplace=True)
for column in ['colour','image', 'brand', 'size','mrp']:
    auditData[column] = auditData[column].str.upper()
auditData['ok_count'] = auditData[['colour','image', 'brand', 'size','mrp']].apply(lambda x: (1 if (x['colour']=='OK' or x['colour']=='NA') else 0 )+ (1 if (x['image']=='OK' or x['image']=='NA') else 0) +
                                                                    (1 if (x['brand']=='OK' or x['brand']=='NA') else 0 ) + (1 if (x['size']=='OK'  or x['size']=='NA') else 0) + 
                                                                    (1 if (x['mrp']=='OK'  or x['mrp']=='NA') else 0), axis=1)
auditData['not_ok_count'] = auditData[['colour','image', 'brand', 'size','mrp']].apply(lambda x: (1 if x['colour']=='NOT OK' else 0 )+ (1 if x['image']=='NOT OK' else 0) +
                                                                    (1 if x['brand']=='NOT OK' else 0 ) + (1 if x['size']=='NOT OK' else 0) + + (1 if x['mrp']=='NOT OK' else 0), axis=1)
auditData['result'] = auditData[['ok_count', 'not_ok_count',]].apply(lambda x: 'Pass' if x['ok_count'] / (x['ok_count'] + x['not_ok_count']) ==1 else 'Fail', axis =1)

In [60]:
auditData[['scanned_date','month', 'year', 'motherhub_name', 'is_tracking_id_available', 'Asset_Name','result']]

,scanned_date,month,year,motherhub_name,is_tracking_id_available,Asset_Name,result
0,2021-09-08,September,2021,MOTHERHUB_SNK,Yes,MotherHub_SNK,Pass
1,2021-09-08,September,2021,MOTHERHUB_SNK,Yes,MotherHub_SNK,Pass
2,2021-09-08,September,2021,MOTHERHUB_SNK,Yes,MotherHub_SNK,Pass
3,2021-09-08,September,2021,MOTHERHUB_ULB,Yes,MotherHub_ULB,Pass
4,2021-09-08,September,2021,MOTHERHUB_ULB,Yes,MotherHub_ULB,Pass
...,...,...,...,...,...,...,...
1646,2022-01-29,January,2022,MOTHERHUB_GGN,Yes,MotherHub_GGN,Fail
1647,2022-01-29,January,2022,MOTHERHUB_GGN,Yes,MotherHub_GGN,Fail
1648,2022-01-29,January,2022,MOTHERHUB_GGN,Yes,MotherHub_GGN,Fail
1649,2022-01-29,January,2022,MOTHERHUB_GGN,Yes,MotherHub_GGN,Fail


In [77]:
auditData[auditData['ok_count']==0][['colour','image', 'brand', 'size','mrp','ok_count', 'not_ok_count']]

,colour,image,brand,size,mrp,ok_count,not_ok_count
47,NOT OK,NOT OK,NOT OK,NOT OK,NOT OK,0,5
83,NOT OK,NOT OK,NOT OK,NOT OK,NOT OK,0,5
583,NOT OK,NOT OK,NOT OK,NOT OK,NOT OK,0,5
895,NOT OK,NOT OK,NOT OK,NOT OK,NOT OK,0,5
896,NOT OK,NOT OK,NOT OK,NOT OK,NOT OK,0,5
897,NOT OK,NOT OK,NOT OK,NOT OK,NOT OK,0,5
898,NOT OK,NOT OK,NOT OK,NOT OK,NOT OK,0,5
899,NOT OK,NOT OK,NOT OK,NOT OK,NOT OK,0,5
900,NOT OK,NOT OK,NOT OK,NOT OK,NOT OK,0,5
901,NOT OK,NOT OK,NOT OK,NOT OK,NOT OK,0,5


In [10]:
audit_df = pd.read_csv(basePath +'Dashboard/data/audit_full_data.csv', low_memory=False)

In [11]:
audit_df

,scanned_date,month,year,weeknum,motherhub_name,zone,is_tracking_id_available,asset,colour,image,brand,size,mrp,result,count
0,2021-09-08,September,2021,37.0,MOTHERHUB_BLR,South 1,Yes,Audit Data,OK,NOT OK,OK,OK,OK,Fail,1
1,2021-09-08,September,2021,37.0,MOTHERHUB_BLR,South 1,Yes,Audit Data,OK,OK,OK,OK,OK,Pass,3
2,2021-09-08,September,2021,37.0,MOTHERHUB_ULB,East,Yes,Audit Data,OK,OK,OK,OK,OK,Pass,11
3,2021-09-14,September,2021,38.0,MOTHERHUB_STV_BTS,West,Yes,Audit Data,OK,OK,OK,OK,OK,Pass,4
4,2021-09-15,September,2021,38.0,MOTHERHUB_HRN,East,Yes,Audit Data,NOT OK,NOT OK,OK,NOT OK,OK,Fail,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2022-01-22,January,2022,4.0,MOTHERHUB_FRK,North,Yes,Audit Data,OK,NOT OK,OK,OK,OK,Fail,4
99,2022-01-22,January,2022,4.0,MOTHERHUB_FRK,North,Yes,Audit Data,OK,OK,OK,OK,NOT OK,Fail,2
100,2022-01-22,January,2022,4.0,MOTHERHUB_FRK,North,Yes,Audit Data,OK,OK,OK,OK,OK,Pass,117
101,2022-01-24,January,2022,5.0,MOTHERHUB_BLR,South 1,Yes,Audit Data,OK,OK,OK,OK,OK,Pass,5


In [20]:
# hub_result_pivot =  
# orphan_pivot = pd.DataFrame(pd.pivot_table(orpdf, index='asset', columns=[date_agg], values=['scanned_timestamp'], aggfunc=len)).reset_index().rename(columns={'scanned_timestamp': 'orphan_count'})
hub_result_pivot = pd.DataFrame(pd.pivot_table(audit_df, index=['motherhub_name'],columns=['result'], fill_value=0, values=['count'], aggfunc=sum)).reset_index().droplevel(0, axis=1).rename(columns={'':'motherhub'})
hub_result_pivot['Total'] = hub_result_pivot.Fail + hub_result_pivot.Pass
hub_result_pivot['Fail_Percentage'] = round((hub_result_pivot.Fail/hub_result_pivot.Total) * 100, 2)
hub_result_pivot

result,motherhub,Fail,Pass,Total,Fail_Percentage
0,MOTHERHUB_ABD,0,3,3,0.00
1,MOTHERHUB_BLR,7,67,74,9.46
2,MOTHERHUB_BNL,0,187,187,0.00
3,MOTHERHUB_CSK,25,0,25,100.00
4,MOTHERHUB_DVD,0,3,3,0.00
5,MOTHERHUB_FRK,17,266,283,6.01
6,MOTHERHUB_GGN,0,10,10,0.00
7,MOTHERHUB_HRN,5,72,77,6.49
8,MOTHERHUB_HYP,0,63,63,0.00
9,MOTHERHUB_JKS,0,14,14,0.00
